In [1]:
using LinearAlgebra

include("./MPS.jl")
using .MPSforQuantum

# MPS

```mps = MPS(C0, eps, normalize='l')```  
C0: state vector  
eps: allowed error  
normalize: 'l' for left normalize (default), 'r' for right normalized

return: mps\[site index\]\[physical index\]\[2x2 array\]

### Left normalized

In [2]:
N = 20
C0 = normalize!(rand(ComplexF64, 2^N))
C0 = zeros(ComplexF64, 2^N)
C0[1] = 1 # '000'

D = 10
eps = 1e-3
mps = MPS(C0, D) # convert to MPS
mps_size(mps) # print the size of MPS

n = 0 # check the state |bin(n)>
println("Restore from MPS: ", restore(mps, n))
println("Original state: ", C0[n + 1])

array 1's size: (1, 2)
array 2's size: (2, 4)
array 3's size: (4, 8)
array 4's size: (8, 10)
array 5's size: (10, 10)
array 6's size: (10, 10)
array 7's size: (10, 10)
array 8's size: (10, 10)
array 9's size: (10, 10)
array 10's size: (10, 10)
array 11's size: (10, 10)
array 12's size: (10, 10)
array 13's size: (10, 10)
array 14's size: (10, 10)
array 15's size: (10, 10)
array 16's size: (10, 10)
array 17's size: (10, 8)
array 18's size: (8, 4)
array 19's size: (4, 2)
array 20's size: (2, 1)
Num of parameters: 2888
2^N: 1048576
Restore from MPS: 1.0 + 0.0im
Original state: 1.0 + 0.0im


In [3]:
mps[20][1] # size (1,1,2)

2×1 Array{Complex{Float64},2}:
 1.0 + 0.0im
 0.0 + 0.0im

### Inner product between same states become Identity matrix for each site
each site is normalized,  
$\sum_{i}A_{i}^{\dagger} A_{i} = I$

In [4]:
site = 7
I_ = mps[site][1]' * mps[site][1] + mps[site][2]' * mps[site][2]
diag(I_)

10-element Array{Complex{Float64},1}:
 1.0 + 0.0im
 1.0 + 0.0im
 1.0 + 0.0im
 1.0 + 0.0im
 1.0 + 0.0im
 1.0 + 0.0im
 1.0 + 0.0im
 1.0 + 0.0im
 1.0 + 0.0im
 1.0 + 0.0im

### Right Normalized

In [11]:
N = 20
C0 = normalize!(rand(ComplexF64, 2^N))
C0 = zeros(ComplexF64, 2^N)
C0[1] = 1 # '010'

D = 10
eps = 1e-3
mps = MPS(C0, D, 'r') # convert to MPS
mps_size(mps) # print the size of MPS

n = 0 # check the state |bin(n)>
println("Restore from MPS: ", restore(mps, n))
println("Original state: ", C0[n + 1])

array 1's size: (1, 2)
array 2's size: (2, 4)
array 3's size: (4, 8)
array 4's size: (8, 10)
array 5's size: (10, 10)
array 6's size: (10, 10)
array 7's size: (10, 10)
array 8's size: (10, 10)
array 9's size: (10, 10)
array 10's size: (10, 10)
array 11's size: (10, 10)
array 12's size: (10, 10)
array 13's size: (10, 10)
array 14's size: (10, 10)
array 15's size: (10, 10)
array 16's size: (10, 10)
array 17's size: (10, 8)
array 18's size: (8, 4)
array 19's size: (4, 2)
array 20's size: (2, 1)
Num of parameters: 2888
2^N: 1048576
Restore from MPS: 1.0 + 0.0im
Original state: 1.0 + 0.0im


In [26]:
C0 = normalize!(rand(ComplexF64, 16))
D = 10
mps = MPS(C0, D, 'r') # convert to MPS
println(mps[1][2] * mps[2][1] * mps[3][1] * mps[4][1])
println(C0)

Complex{Float64}[0.1343376505913656 + 0.01774765179570367im]
Complex{Float64}[0.011740178782516058 + 0.003983272029447662im, 0.13433765059136568 + 0.017747651795703732im, 0.08611102076840618 + 0.2385299811327732im, 0.1712517998216532 + 0.24101229937136617im, 0.23458608960616237 + 0.09360925889589386im, 0.2631730486470682 + 0.2582093308282593im, 0.010045474074949774 + 0.22645099352827477im, 0.13768451079774333 + 0.0023690937291059467im, 0.14186504624813817 + 0.3082128462107923im, 0.2669604798382588 + 0.1060518353446732im, 0.03357332877822472 + 0.1935553461919171im, 0.06888535767042009 + 0.2684970815396278im, 0.04306551923498472 + 0.08644639868993309im, 0.1741213989188889 + 0.06161857309341674im, 0.30527252710661856 + 0.05121449971407059im, 0.07813235428177782 + 0.3182229114165836im]


## SVDを用いたMPS作成の簡単な例
行列```tmp```を特異値分解により3つの行列に分ける。  
3つの行列をさらに2つの行列に分ける(=physical index 0 & 1)。  
分解前の行列の特定のインデックス番号をバイナリ値で表し、それに対応したphysical indexを持つ各siteの行列同士をかけることで、分解前の行列要素を復元できる。

### Left normalized

In [53]:
tmp = svd(reshape([1, 2, 3, 4, 5, 6, 7, 8], 2, 4))
A1 = tmp.U
C1 = diagm(tmp.S) * tmp.Vt
println(transpose(tmp.U[1, :]) * diagm(tmp.S) * tmp.Vt)
println(transpose(tmp.U[2, :]) * diagm(tmp.S) * tmp.Vt)

[1.0000000000000007 3.0 5.000000000000002 7.000000000000001]
[2.000000000000006 4.0000000000000036 6.000000000000005 8.000000000000007]


In [54]:
tmp = svd(reshape(C1, 4, 2))
A2 = tmp.U
A3 = diagm(tmp.S) * tmp.Vt

2×2 Array{Float64,2}:
  5.3519   13.1824
 -1.16498   0.472968

In [55]:
transpose(A1[2, :]) * A2[1:2, :] * transpose(A3)[1, :] # '001

2.0000000000000058

### Right normalized

In [12]:
tmp = svd(reshape([i for i in 1:16], 8, 2))
A1 = tmp.U * diagm(tmp.S)
C1 = tmp.Vt
arr = cat(A1[1:4, :], A1[5:8, :], dims=2)
tmp2 = svd(arr)
A2 = tmp2.U * diagm(tmp2.S)
C2 = tmp2.Vt
arr = cat(A2[1:2, :], A2[3:4, :], dims=2)
tmp3 = svd(arr)
C4 = tmp3.U * diagm(tmp3.S)
C3 = tmp3.Vt
println(size(C4), size(C3), size(C2), size(C1))
transpose(C4[2,:]) * C3[:, 1:4] * C2[:, 1:2] * C1[:, 1] # '0001'  -> 2nd element

(2, 2)(2, 8)(4, 4)(2, 2)


2.0000000000000053

# MPO

期待値を求めたいハミルトニアン  
簡単のため、3 siteの系を考える  

$Z_1 Z_2 + Z_2 Z_3$

MPOは下記のように組む。  
site数と同じ3つの行列をする。  
行列同士の積を計算すると、上記のハミルトニアンになるように要素を決めている。


$$
\left[\begin{array}{c}
    Z_1 & I_1\\
\end{array}\right]
\left[\begin{array}{c}
    Z_2 & 0\\
    0 & Z_2\\
\end{array}\right]
\left[\begin{array}{c}
    I_3 \\
    Z_3 \\
\end{array}\right] \quad
$$

In [12]:
pauliZ = convert(Array{ComplexF64,2}, [1 0; 0 -1])
pauliI = convert(Array{ComplexF64,2}, [1 0; 0 1])
zero = convert(Array{ComplexF64,2}, [0 0; 0 0])

O = []
push!(O, dstack((pauliZ, pauliI)))
push!(O, ddstack([(pauliZ, zero), (zero, pauliZ)]))
push!(O, dstack((pauliI, pauliZ)))

# O[site], [phys1, phys2, i, j]

O[1]

2×2×2 Array{Complex{Float64},3}:
[:, :, 1] =
 1.0+0.0im   0.0+0.0im
 0.0+0.0im  -1.0+0.0im

[:, :, 2] =
 1.0+0.0im  0.0+0.0im
 0.0+0.0im  1.0+0.0im

# General inner products
2つのMPSにそれぞれ異なる量子ゲートをかけてから内積を計算する

In [13]:
Hadamard = convert(Array{ComplexF64,2}, [1 1; 1 -1] ./ sqrt(2))
pauliX = convert(Array{ComplexF64,2}, [0 1; 1 0])

2×2 Array{Complex{Float64},2}:
 0.0+0.0im  1.0+0.0im
 1.0+0.0im  0.0+0.0im

In [14]:
N = 3
C0 = zeros(ComplexF64, 2^N)
C0[1] = 1 # '000'
# C0[8] = 1 # '111'

D = 10
eps = 1e-3
mps1 = MPS(C0, eps)
mps1 = OneQubitGate(mps1, Hadamard, 0)
mps1 = OneQubitGate(mps1, Hadamard, 1)
mps1 = OneQubitGate(mps1, Hadamard, 2)

mps2 = MPS(C0, eps)
mps2 = OneQubitGate(mps2, pauliX, 0)
mps1

3-element Array{Any,1}:
 Array{Complex{Float64},2}[[0.7071067811865475 + 0.0im], [0.7071067811865475 + 0.0im]]
 Array{Complex{Float64},2}[[0.7071067811865475 + 0.0im], [0.7071067811865475 + 0.0im]]
 Array{Complex{Float64},2}[[0.7071067811865475 + 0.0im], [0.7071067811865475 + 0.0im]]

In [16]:
inner_product(mps1, mps2)

1×1 Array{Complex{Float64},2}:
 0.3535533905932737 + 0.0im

### Bell state

In [17]:
Hadamard = convert(Array{ComplexF64,2}, [1 1; 1 -1] ./ sqrt(2))

N = 20
C0 = zeros(ComplexF64, 2^N)
C0[1] = 1 # '000'

D = 10
eps = 1e-3

mps1 = MPS(C0, eps)
println("Initial MPS")
mps1 = OneQubitGate(mps1, Hadamard, 0) # -> '001' & '000'
mps_size(mps1)

arr = CX(mps1, 0, eps) # 今の所、i と i+1の間での計算のみ。 多分swapゲートが必要
println("\nBell state MPS")
mps_size(arr)
println()
for i=0:3
    res = restore(arr, i) # MPS で表された状態から、| bin(i) > の係数を計算する関数
    println("|", bitstring(i)[end - 1:end], ">: ",  res)
end

Initial MPS
array 1's size: (1, 1)
array 2's size: (1, 1)
array 3's size: (1, 1)
array 4's size: (1, 1)
array 5's size: (1, 1)
array 6's size: (1, 1)
array 7's size: (1, 1)
array 8's size: (1, 1)
array 9's size: (1, 1)
array 10's size: (1, 1)
array 11's size: (1, 1)
array 12's size: (1, 1)
array 13's size: (1, 1)
array 14's size: (1, 1)
array 15's size: (1, 1)
array 16's size: (1, 1)
array 17's size: (1, 1)
array 18's size: (1, 1)
array 19's size: (1, 1)
array 20's size: (1, 1)
Num of parameters: 40
2^N: 1048576

Bell state MPS
array 1's size: (1, 2)
array 2's size: (2, 1)
array 3's size: (1, 1)
array 4's size: (1, 1)
array 5's size: (1, 1)
array 6's size: (1, 1)
array 7's size: (1, 1)
array 8's size: (1, 1)
array 9's size: (1, 1)
array 10's size: (1, 1)
array 11's size: (1, 1)
array 12's size: (1, 1)
array 13's size: (1, 1)
array 14's size: (1, 1)
array 15's size: (1, 1)
array 16's size: (1, 1)
array 17's size: (1, 1)
array 18's size: (1, 1)
array 19's size: (1, 1)
array 20's size: (1

## 確認
状態ベクトルでBell state作ってからMPSに変換して同じになってるか確認してみた。

In [18]:
N = 2
C0 = zeros(ComplexF64, 2^N)
C0[1] = 0.7071067811865475
C0[4] = 0.7071067811865475

eps = 1e-3

mps1 = MPS(C0, eps)
mps_size(mps1)
for i=0:3
    res = restore(mps1, i)
    println("|", bitstring(i)[end - 1:end], ">: ",  res)
end

array 1's size: (1, 2)
array 2's size: (2, 1)
Num of parameters: 8
2^N: 4
|00>: 0.7071067811865475 + 0.0im
|01>: 0.0 + 0.0im
|10>: 0.0 + 0.0im
|11>: 0.7071067811865475 + 0.0im


# Expectation value

### 3量子ビットハイゼンベルグモデル

$Z_1 Z_2 + Z_2 Z_3$

MPO表現は

$$
\left[\begin{array}{c}
    Z_1 & I_1\\
\end{array}\right]
\left[\begin{array}{c}
    Z_2 & 0\\
    0 & Z_2\\
\end{array}\right]
\left[\begin{array}{c}
    I_3\\
    Z_3
\end{array}\right] \quad
$$

具体的な計算は、各siteでまずMPSx2,MPOをcontructしてから、site同士でcontract。  

$$
F^{[i]}_{a_i,b_i,a'_i} = \sum_{\sigma_{i},a_{i-1}}(A^{[i]\sigma_i\dagger})_{a_i,a_{i-1}}
\biggl(\sum_{\sigma'_{i},b_{i-1}}W^{[i]\sigma_i \sigma'_i}_{b_{i-1},b_i} 
\biggl(\sum_{a'_{i-1}}F^{[i-1]}_{a_{i-1},b_{i-1},a'_{i-1}} A^{[i]\sigma'_i}_{a'_{i-1},a'_i} \biggr) \biggr)
$$

上記MPOの説明と全く同じようにMPO "O" を用意

In [21]:
pauliZ = convert(Array{ComplexF64,2}, [1 0; 0 -1])
pauliI = convert(Array{ComplexF64,2}, [1 0; 0 1])
zero = convert(Array{ComplexF64,2}, [0 0; 0 0])

O = []
push!(O, dstack((pauliZ, pauliI)))
push!(O, ddstack([(pauliZ, zero), (zero, pauliZ)]))
push!(O, dstack((pauliI, pauliZ)))

3-element Array{Any,1}:
 Complex{Float64}[1.0 + 0.0im 0.0 + 0.0im; 0.0 + 0.0im -1.0 + 0.0im]

Complex{Float64}[1.0 + 0.0im 0.0 + 0.0im; 0.0 + 0.0im 1.0 + 0.0im]
 Complex{Float64}[1.0 + 0.0im 0.0 + 0.0im; 0.0 + 0.0im -1.0 + 0.0im]

Complex{Float64}[0.0 + 0.0im 0.0 + 0.0im; 0.0 + 0.0im 0.0 + 0.0im]

Complex{Float64}[0.0 + 0.0im 0.0 + 0.0im; 0.0 + 0.0im 0.0 + 0.0im]

Complex{Float64}[1.0 + 0.0im 0.0 + 0.0im; 0.0 + 0.0im -1.0 + 0.0im]
 Complex{Float64}[1.0 + 0.0im 0.0 + 0.0im; 0.0 + 0.0im 1.0 + 0.0im]

Complex{Float64}[1.0 + 0.0im 0.0 + 0.0im; 0.0 + 0.0im -1.0 + 0.0im]

### 3量子ビットの計算基底8通りについて、ハミルトニアンの期待値を計算

In [22]:
N = 3
eps = 1e-3
D = 10

for i in 1:8
    C0 = zeros(ComplexF64, 2^N)
    C0[i] = 1
    mps = MPS(C0, eps)
    expc = expectation(mps, O)
    println(expc)
end

2.0 + 0.0im
0.0 + 0.0im
-2.0 + 0.0im
0.0 + 0.0im
0.0 + 0.0im
-2.0 + 0.0im
0.0 + 0.0im
2.0 + 0.0im


### 通常の量子状態ベクトルを用いた期待値計算

In [47]:
function tensordot(x::Array, y::Array)
    lx = size(x)
    ly = size(y)
    res = zeros(lx[1] * ly[1], lx[2] * ly[2])
    for i in 1:lx[1]
        for j in 1:lx[2]
            res[ (1+(i-1)*ly[1]):(ly[1]+(i-1)*ly[1]), (1+(j-1)*ly[2]):(ly[2]+(j-1)*ly[2]) ] = x[i,j] * y
        end
    end
    return res
end

tensordot (generic function with 1 method)

In [51]:
# Hamiltonian
H = tensordot(tensordot(pauliZ, pauliZ), pauliI) + tensordot(tensordot(pauliI, pauliZ), pauliZ)

for i in 1:8
    vec = zeros(8)
    vec[i] = 1
    expc = transpose(vec) * H * vec
    println(expc)
end

2.0
0.0
-2.0
0.0
0.0
-2.0
0.0
2.0


### 4量子ビット ハイゼンベルグモデルの場合

$Z_1 Z_2 + Z_2 Z_3 + Z_3 Z_4$

In [8]:
pauliZ = convert(Array{ComplexF64,2}, [1 0; 0 -1])
pauliI = convert(Array{ComplexF64,2}, [1 0; 0 1])
zero = convert(Array{ComplexF64,2}, [0 0; 0 0])

O = []
push!(O, dstack((pauliZ, pauliI, pauliI)) )
push!(O, ddstack( [(pauliZ, zero, zero), (zero, pauliZ, zero), (zero, zero, pauliI)] ))
push!(O, ddstack( [(pauliI, zero, zero), (zero, pauliZ, zero), (zero, zero, pauliZ)] ))
push!(O, dstack((pauliI, pauliI, pauliZ)))
O[4]

2×2×3 Array{Complex{Float64},3}:
[:, :, 1] =
 1.0+0.0im  0.0+0.0im
 0.0+0.0im  1.0+0.0im

[:, :, 2] =
 1.0+0.0im  0.0+0.0im
 0.0+0.0im  1.0+0.0im

[:, :, 3] =
 1.0+0.0im   0.0+0.0im
 0.0+0.0im  -1.0+0.0im

### 計算基底16通りについて期待値を計算

### Left normalized

In [31]:
N = 4
eps = 1e-3
D = 10

for i in 1:16
    C0 = zeros(ComplexF64, 2^N)
    C0[i] = 1 # computational basis
    #C0 = normalize!(rand(ComplexF64, 2^N))
    mps = MPS(C0, D)
    # println(restore(mps, 4))
    expc = expectation(mps, O)
    println(expc)
end

3.0 + 0.0im
1.0 + 0.0im
-1.0 + 0.0im
1.0 + 0.0im
-1.0 + 0.0im
-3.0 + 0.0im
-1.0 + 0.0im
1.0 + 0.0im
1.0 + 0.0im
-1.0 + 0.0im
-3.0 + 0.0im
-1.0 + 0.0im
1.0 + 0.0im
-1.0 + 0.0im
1.0 + 0.0im
3.0 + 0.0im


### Right normalized

Normalizeの違いによって期待値は変わらないため、Left normalizedと同じ結果となるのが正しい

In [30]:
N=4
for i in 1:16
    C0 = zeros(ComplexF64, 2^N)
    C0[i] = 1 # computational basis
    #C0 = normalize!(rand(ComplexF64, 2^N))
    mps = MPS(C0, D, 'r')
    # println(restore(mps, 4))
    expc = expectation(mps, O)
    println(expc)
end

3.0 + 0.0im
1.0 + 0.0im
-1.0 + 0.0im
1.0 + 0.0im
-1.0 + 0.0im
-3.0 + 0.0im
-1.0 + 0.0im
1.0 + 0.0im
1.0 + 0.0im
-1.0 + 0.0im
-3.0 + 0.0im
-1.0 + 0.0im
1.0 + 0.0im
-1.0 + 0.0im
1.0 + 0.0im
3.0 + 0.0im
